In [126]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Define a function that imports all the data from the provided csv files

In [125]:
csv_paths = glob.glob("data/*.csv")
csv_filenames = [os.path.splitext(os.path.basename(x))[0] for x in csv_paths]

def load_data_from_csv():
    """Loads the csv files to pandas DataFrames and saves
    them in a dictionary with their filenames as keys"""
    data = dict()
    for csv_filename, csv_path in zip(csv_filenames, csv_paths):
        data[csv_filename] = pd.read_csv(csv_path)
    return data

# Load the data to a Dictionary
data = load_data_from_csv()

Inspect the data, the schema is shown below:

<img src="docs/schema.png">

In [64]:
for key, values in data.items():
    print(key, "\n")
    print(values.head(), "\n")
    print(values.describe(), "\n")

app_events 

   event_id               app_id  is_installed  is_active
0         2  5927333115845830913             1          1
1         2 -5720078949152207372             1          0
2         2 -1633887856876571208             1          0
3         2  -653184325010919369             1          1
4         2  8693964245073640147             1          1 

           event_id        app_id  is_installed     is_active
count  3.247307e+07  3.247307e+07    32473067.0  3.247307e+07
mean   1.625564e+06  1.182779e+18           1.0  3.921094e-01
std    9.384682e+05  5.360173e+18           0.0  4.882209e-01
min    2.000000e+00 -9.221157e+18           1.0  0.000000e+00
25%    8.134720e+05 -3.474568e+18           1.0  0.000000e+00
50%    1.626907e+06  1.387044e+18           1.0  0.000000e+00
75%    2.441106e+06  6.043001e+18           1.0  1.000000e+00
max    3.252948e+06  9.222488e+18           1.0  1.000000e+00 

app_labels 

                app_id  label_id
0  7324884708820027918       25

First replace the chinese brands with their english names

In [127]:
# Join chinese brand names to their english equivalents
phone_brand_device_model = pd.merge(data["phone_brand_device_model"],
                                    data["chinese_to_english_brand"],
                                    left_on="phone_brand", right_on="chinese_brand", how="left")

# Fill nans for brands already in english
phone_brand_device_model["brand"] =\
    phone_brand_device_model["brand"].where(phone_brand_device_model["brand"].str.isalpha(),
                                                    phone_brand_device_model["phone_brand"].str.lower())
# We will just work with device brand for the time being
device_data = phone_brand_device_model[["device_id", "brand"]]
device_data

,device_id,brand
0,-8890648629457979026,xiaomi
1,1277779817574759137,xiaomi
2,5137427614288105724,samsung
3,3669464369358936369,sugar
4,-5019277647504317457,samsung
...,...,...
187240,7979541072208733273,xiaomi
187241,-187404680852357705,xiaomi
187242,-2718274279595622821,xiaomi
187243,3098391762071677791,vivo


We'll proceed to join the device brand data to the test data

In [135]:
data["gender_age_train"]["group"].sort_values().unique()

array(['F23-', 'F24-26', 'F27-28', 'F29-32', 'F33-42', 'F43+', 'M22-',
       'M23-26', 'M27-28', 'M29-31', 'M32-38', 'M39+'], dtype=object)

In [128]:
combined_data = data["gender_age_train"]
combined_data = combined_data.merge(device_data, on="device_id")
combined_data

,device_id,gender,age,group,brand
0,-8076087639492063270,M,35,M32-38,xiaomi
1,-2897161552818060146,M,35,M32-38,xiaomi
2,-8260683887967679142,M,35,M32-38,xiaomi
3,-4938849341048082022,M,30,M29-31,xiaomi
4,245133531816851882,M,30,M29-31,xiaomi
...,...,...,...,...,...
74834,4682031842235089751,M,30,M29-31,xiaomi
74835,-9178703742877135986,M,30,M29-31,xiaomi
74836,180946546684162312,M,20,M22-,xiaomi
74837,1390702386071991851,M,37,M32-38,huawei


Put together the event data before merging all

Merge 